In [1]:
import pandas as pd

idle_time_data = pd.read_csv('../data/df_points/df_points_18_21_class.csv')
from sklearn.preprocessing import StandardScaler

TargetVariable = ['idle_time_class']
Predictors = ['bike_id', 'lat', 'lng', 'temp', 'rain', 'snow', 'dt_start', 'hex_enc', 'start_min', 'month', 'day']

X = idle_time_data[Predictors].values
y = idle_time_data[TargetVariable].values

PredictorScaler = StandardScaler()
PredictorScalerFit = PredictorScaler.fit(X)
X = PredictorScalerFit.transform(X)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, shuffle=False)
### shuffle/inorder train and test sets:
# , shuffle=False
# , random_state=42

In [4]:
pd.DataFrame(y_test).value_counts()

2    68622
1    66954
3    63246
4    55894
dtype: int64

In [8]:
sweep_configuration = {
    "project": "RandomForestClass",
    "name": "RFC-sweep-split-inorder",
    "metric": {"name": "accuracy", "goal": "maximize"},
    "method": "random",
    "parameters": {
        "splitter": {
            "values": ['best','random']
        },
        "criterion": {
            "values": ['entropy','gini']
        },
        "max_depth": {
            "values": [2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 120, 140, 160, 180, 200, None]
        },
        "max_features": {
            "values": ['auto', 'sqrt', 'log2']
        },
        "min_samples_leaf": {
            "values": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 14, 16, 18, 20, 25, 30]
        },
        "min_samples_split": {
            "values": [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 15, 18, 20, 25, 30]
        }
    }
}

In [9]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import zero_one_loss, accuracy_score
from sklearn.model_selection import cross_validate, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import wandb
import wandb

def my_train_func():
    wandb.init()

    _splitter= wandb.config.splitter
    _criterion = wandb.config.criterion
    _max_depth = wandb.config.max_depth
    _max_features = wandb.config.max_features
    _min_samples_leaf = wandb.config.min_samples_leaf
    _min_samples_split = wandb.config.min_samples_split

    model = DecisionTreeClassifier(criterion=_criterion,
                                   max_depth=_max_depth,
                                   splitter=_splitter,
                                   max_features=_max_features,
                                   min_samples_leaf=_min_samples_leaf,
                                   min_samples_split=_min_samples_split)

    model.fit(X_train,y_train.ravel())
    y_pred = model.predict(X_test)

    score_training = model.score(X_train, y_train.ravel())
    score_validation = model.score(X_test, y_test.ravel())

    acc = zero_one_loss(y_test.ravel(), y_pred.ravel())
    loss = accuracy_score(y_test.ravel(), y_pred.ravel())

    wandb.sklearn.plot_feature_importances(model, Predictors)

    wandb.log({"accuracy": acc})
    wandb.log({"conf_matrix": wandb.plot.confusion_matrix(y_true=y_test.ravel(), preds=y_pred.ravel())})
    wandb.log({"feature_imp": wandb.sklearn.plot_feature_importances(model, Predictors)})
    wandb.log({"loss": loss})
    wandb.log({"score_training":score_training, "score_validation":score_validation})


In [10]:
# INIT SWEEP
sweep_id_rfc = wandb.sweep(sweep_configuration, project="test")
# RUN SWEEP
wandb.agent(sweep_id_rfc, function=my_train_func)

Create sweep with ID: 02ts6cxp
Sweep URL: https://wandb.ai/jonathanweske/test/sweeps/02ts6cxp


wandb: Agent Starting Run: 40tpg48b with config:
wandb: 	criterion: entropy
wandb: 	max_depth: 10
wandb: 	max_features: sqrt
wandb: 	min_samples_leaf: 20
wandb: 	min_samples_split: 4
wandb: 	splitter: best


accuracy,▁
loss,▁
score_training,▁
score_validation,▁
accuracy,0.55111
loss,0.44889
score_training,0.48927
score_validation,0.44889


wandb: Agent Starting Run: d0ys0mtu with config:
wandb: 	criterion: gini
wandb: 	max_depth: 8
wandb: 	max_features: sqrt
wandb: 	min_samples_leaf: 6
wandb: 	min_samples_split: 2
wandb: 	splitter: random


accuracy,▁
loss,▁
score_training,▁
score_validation,▁
accuracy,0.5722
loss,0.4278
score_training,0.4586
score_validation,0.4278


...

wandb: Agent Starting Run: 64k39pkj with config:
wandb: 	criterion: gini
wandb: 	max_depth: 70
wandb: 	max_features: auto
wandb: 	min_samples_leaf: 7
wandb: 	min_samples_split: 6
wandb: 	splitter: random


accuracy,▁
loss,▁
score_training,▁
score_validation,▁
accuracy,0.55791
loss,0.44209
score_training,0.57158
score_validation,0.44209


wandb: Agent Starting Run: 6pl0frli with config:
wandb: 	criterion: gini
wandb: 	max_depth: 10
wandb: 	max_features: auto
wandb: 	min_samples_leaf: 30
wandb: 	min_samples_split: 4
wandb: 	splitter: random


accuracy,▁
loss,▁
score_training,▁
score_validation,▁
accuracy,0.59637
loss,0.40363
score_training,0.42026
score_validation,0.40363


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: mzd30alp with config:
wandb: 	criterion: gini
wandb: 	max_depth: 40
wandb: 	max_features: log2
wandb: 	min_samples_leaf: 2
wandb: 	min_samples_split: 15
wandb: 	splitter: best
wandb: Ctrl + C detected. Stopping sweep.
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/usr/local/Caskroom/miniconda/base/envs/geobike/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/usr/local/Caskroom/miniconda/base/envs/geobike/lib/python3.9/multiprocessing/spawn.py", line 125, in _main
    prepare(preparation_data)
  File "/usr/local/Caskroom/miniconda/base/envs/geobike/lib/python3.9/multiprocessing/spawn.py", line 234, in prepare
    _fixup_main_from_name(data['init_main_from_name'])
  File "/usr/local/Caskroom/miniconda/base/envs/geobike/lib/python3.9/multiprocessing/spawn.py", line 258, in _fixup_main_from_name
    main_